In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
os.environ["TOKENIZERS_PARALLELISM"] = "false"
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/nlp-itmo-exercise-3/submission_template.txt
/kaggle/input/nlp-itmo-exercise-3/ground_truth.txt
/kaggle/input/nlp-itmo-exercise-3/archive/train_10.json
/kaggle/input/nlp-itmo-exercise-3/archive/emb_model.txt
/kaggle/input/nlp-itmo-exercise-3/archive/emb_eng.txt
/kaggle/input/nlp-itmo-exercise-3/archive/train_50.json
/kaggle/input/nlp-itmo-exercise-3/archive/train_10000.json
/kaggle/input/nlp-itmo-exercise-3/archive/submission_template.txt
/kaggle/input/nlp-itmo-exercise-3/archive/imdb_dataset.csv
/kaggle/input/nlp-itmo-exercise-3/archive/train_100.json
/kaggle/input/nlp-itmo-exercise-3/archive/train_5.json
/kaggle/input/nlp-itmo-exercise-3/archive/test.csv
/kaggle/input/nlp-itmo-exercise-3/archive/train_1000.json
/kaggle/input/nlp-itmo-exercise-3/archive/train_20.json
/kaggle/input/nlp-itmo-exercise-3/archive/train_5000.json


In [2]:
!ls /kaggle/working/lightning_logs

ls: cannot access '/kaggle/working/lightning_logs': No such file or directory


In [3]:
!ls /kaggle/working/lightning_logs/version_3/checkpoints/epoch=0-step=11243.ckpt

ls: cannot access '/kaggle/working/lightning_logs/version_3/checkpoints/epoch=0-step=11243.ckpt': No such file or directory


In [4]:
!nvidia-smi

Thu Nov  9 14:55:22 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.161.03   Driver Version: 470.161.03   CUDA Version: 11.4     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   34C    P0    26W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [5]:
%pip install lightning

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 19.2 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [6]:
%pip install torchmetrics

Note: you may need to restart the kernel to use updated packages.


In [7]:
import torch
from torch import optim
from torch.utils.data import DataLoader
from torchmetrics.functional.text import bleu_score
from transformers import AutoTokenizer, AutoModelForCausalLM
import lightning.pytorch as pl
from datasets import load_dataset

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [8]:
class GenModel(pl.LightningModule):
    def __init__(self, model_name_or_path):
        super().__init__()
        
        self.tokenizer = AutoTokenizer.from_pretrained(model_name_or_path, truncation_side="left", padding_side='left')
        self.tokenizer.pad_token = self.tokenizer.eos_token
        self.model = AutoModelForCausalLM.from_pretrained(model_name_or_path, pad_token_id=self.tokenizer.eos_token_id)
    
    def training_step(self, batch, batch_idx):
        loss = self.model(input_ids=batch["input_ids"].to(device="cuda"), attention_mask=batch["attention_mask"].to(device="cuda"), labels=batch["input_ids"].to(device="cuda")).loss
        self.log("train_loss", loss.item(), prog_bar=True)
        return loss
    
    def validation_step(self, batch, batch_idx):
        val_loss = self.model(input_ids=batch["input_ids"].to(device="cuda"), attention_mask=batch["attention_mask"].to(device="cuda"), labels=batch["input_ids"].to(device="cuda")).loss
        self.log("val_loss", val_loss.item(), prog_bar=True)
        return val_loss
    
    def test_step(self, batch, batch_idx):
        outputs = self.model.generate(batch["input_ids"].to(device="cuda"), attention_mask=batch["attention_mask"].to(device="cuda"))
        preds = []
        for i in range(len(outputs)):
            pred = self.tokenizer.decode(outputs[i], skip_special_tokens=True)
            preds.append(pred)
        
        bleu = bleu_score(preds, batch["summaries"], n_gram=1)
        self.log("bleu", bleu.item(), prog_bar=True)
        return bleu
        
    def configure_optimizers(self):
        optimizer = optim.Adam(self.parameters(), lr=2e-5)
        return optimizer

In [9]:
class GenDataModule(pl.LightningDataModule):
    def __init__(self, model_name_or_path, ds_name):
        super().__init__()
        self.tokenizer = AutoTokenizer.from_pretrained(model_name_or_path, truncation_side="left", padding_side='left')
        self.tokenizer.pad_token = self.tokenizer.eos_token
        self.ds_name = ds_name
        
    def prepare_data(self):
        self.ds = load_dataset(self.ds_name, name="ru")
        self.ds["train"] = self.ds["train"].select(list(range(10_000))).map(self.tokenize)
        self.ds["validation"] = self.ds["validation"].select(list(range(2_000))).map(self.tokenize)
        self.ds["test"] = self.ds["test"].select(list(range(2_000))).map(self.test_tokenize)
        
    def train_dataloader(self):
        train_split = self.ds["train"]
        return DataLoader(train_split, batch_size=4, collate_fn=self.collate, num_workers=3)
    
    def val_dataloader(self):
        val_split = self.ds["validation"]
        return DataLoader(val_split, batch_size=4, collate_fn=self.collate, num_workers=3)
    
    def test_dataloader(self):
        test_split = self.ds["test"]
        return DataLoader(test_split, batch_size=4, collate_fn=self.collate_test, num_workers=3)
    
    def tokenize(self, row):
        row = self.tokenizer(f'{row["source"]} summary: {row["target"]}', max_length=512, padding='max_length', truncation=True, return_tensors="pt")
        return row
    
    def test_tokenize(self, row):
        row = self.tokenizer(f'{row["source"]} summary:', max_length=512, padding='max_length', truncation=True, return_tensors="pt")
        return row
    
    def collate(self, samples):
        input_ids = torch.stack([torch.tensor(s["input_ids"]).squeeze() for s in samples])
        attention_mask = torch.stack([torch.tensor(s["attention_mask"]).squeeze() for s in samples])
        return {"input_ids": input_ids, "attention_mask": attention_mask}
    
    def collate_test(self, samples):
        input_ids = torch.stack([torch.tensor(s["input_ids"]).squeeze() for s in samples])
        attention_mask = torch.stack([torch.tensor(s["attention_mask"]).squeeze() for s in samples])
        summaries = [row["target"] for row in samples]
        return {"input_ids": input_ids, "attention_mask": attention_mask, "summaries": summaries}
        

In [10]:
gen_model = GenModel("ai-forever/rugpt3small_based_on_gpt2")

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [11]:
gen_dm = GenDataModule("ai-forever/rugpt3small_based_on_gpt2", "GEM/wiki_lingua")

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [12]:
trainer = pl.Trainer(max_time="00:04:00:00")
trainer.test(model=gen_model, datamodule=gen_dm)

INFO: GPU available: True (cuda), used: True
INFO: TPU available: False, using: 0 TPU cores
INFO: IPU available: False, using: 0 IPUs
INFO: HPU available: False, using: 0 HPUs


Generating train split:   0%|          | 0/35313 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/4984 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/9962 [00:00<?, ? examples/s]

Generating sampled_validation split:   0%|          | 0/3000 [00:00<?, ? examples/s]

...

INFO: LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing: |          | 0/? [00:00<?, ?it/s]

/opt/conda/lib/python3.10/site-packages/transformers/generation/utils.py:1260: UserWarning: Using the model-agnostic default `max_length` (=20) to control thegeneration length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/transformers/generation/utils.py:1268: UserWarning: Input length of input_ids is 512, but `max_length` is set to 20. This can lead to unexpected behavior. You should consider increasing `max_new_tokens`.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/lightning/pytorch/utilities/data.py:77: Trying to infer the `batch_size` from an ambiguous collection. The batch size we found is 4. To avoid any miscalculations, use `self.log(..., batch_size=batch_size)`.


┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│           bleu            │   0.041088927537202835    │
└───────────────────────────┴───────────────────────────┘

[{'bleu': 0.041088927537202835}]

In [13]:
trainer.fit(model=gen_model, datamodule=gen_dm)

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/10000 [00:00<?, ?ex/s]

  0%|          | 0/2000 [00:00<?, ?ex/s]

  0%|          | 0/2000 [00:00<?, ?ex/s]

INFO: LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO: 
  | Name  | Type            | Params
------------------------------------------
0 | model | GPT2LMHeadModel | 125 M 
------------------------------------------
125 M     Trainable params
0         Non-trainable params
125 M     Total params
500.926   Total estimated model params size (MB)


...

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

INFO: Time limit reached. Elapsed time is 4:00:00. Signaling Trainer to stop.


Validation: |          | 0/? [00:00<?, ?it/s]

In [14]:
trainer.test(model=gen_model, datamodule=gen_dm)

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/10000 [00:00<?, ?ex/s]

  0%|          | 0/2000 [00:00<?, ?ex/s]

  0%|          | 0/2000 [00:00<?, ?ex/s]

INFO: LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing: |          | 0/? [00:00<?, ?it/s]

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│           bleu            │    0.04109092429280281    │
└───────────────────────────┴───────────────────────────┘

[{'bleu': 0.04109092429280281}]

In [15]:
text = """– Вам известно, – сказал аббат, – что я был секретарем, доверенным другом кардинала Спада, 
последнего представителя древнего рода. Этому достойному вельможе я обязан всем счастьем, которое 
я знал в жизни. Он не был богат, хотя богатства его рода стали притчей во языцех, и мне часто приходилось 
слышать выражение: «Богат, как Спада» И он и молва жили за счет этих пресловутых богатств. Его дворец 
был раем для меня. Я учил его племянников, которые потом скончались, и когда он остался один на свете, 
то я отплатил ему беззаветной преданностью за все, что он для меня сделал в продолжение десяти лет.

В доме кардинала от меня не было тайн; не раз видел я, как он усердно перелистывает старинные книги и 
жадно роется в пыли фамильных рукописей. Когда я как-то упрекнул его за бесполезные бессонные ночи, 
после которых он впадал в болезненное уныние, он взглянул на меня с горькой улыбкой и раскрыл передо мною 
историю города Рима. В этой книге, в двадцатой главе жизнеописания папы Александра Шестого, я прочел 
следующие строки, навсегда оставшиеся в моей памяти."""

In [16]:
inputs = gen_model.tokenizer(f'{text} summary:', max_length=512, padding='max_length', truncation=True, return_tensors="pt")

In [17]:
gen_model.to("cuda")
outputs = gen_model.model.generate(input_ids=inputs["input_ids"].to(device="cuda"), attention_mask=inputs["attention_mask"].to(device="cuda"))

In [18]:
pred = gen_model.tokenizer.decode(outputs[0], skip_special_tokens=True)

In [19]:
pred

'– Вам известно, – сказал аббат, – что я был секретарем, доверенным другом кардинала Спада, \nпоследнего представителя древнего рода. Этому достойному вельможе я обязан всем счастьем, которое \nя знал в жизни. Он не был богат, хотя богатства его рода стали притчей во языцех, и мне часто приходилось \nслышать выражение: «Богат, как Спада» И он и молва жили за счет этих пресловутых богатств. Его дворец \nбыл раем для меня. Я учил его племянников, которые потом скончались, и когда он остался один на свете, \nто я отплатил ему беззаветной преданностью за все, что он для меня сделал в продолжение десяти лет.\n\nВ доме кардинала от меня не было тайн; не раз видел я, как он усердно перелистывает старинные книги и \nжадно роется в пыли фамильных рукописей. Когда я как-то упрекнул его за бесполезные бессонные ночи, \nпосле которых он впадал в болезненное уныние, он взглянул на меня с горькой улыбкой и раскрыл передо мною \nисторию города Рима. В этой книге, в двадцатой главе жизнеописания папы 

In [20]:
inputs = gen_model.tokenizer(pred, max_length=512, padding='max_length', truncation=True, return_tensors="pt")
outputs = gen_model.model.generate(input_ids=inputs["input_ids"].to(device="cuda"), attention_mask=inputs["attention_mask"].to(device="cuda"))
pred = gen_model.tokenizer.decode(outputs[0], skip_special_tokens=True)
pred

'– Вам известно, – сказал аббат, – что я был секретарем, доверенным другом кардинала Спада, \nпоследнего представителя древнего рода. Этому достойному вельможе я обязан всем счастьем, которое \nя знал в жизни. Он не был богат, хотя богатства его рода стали притчей во языцех, и мне часто приходилось \nслышать выражение: «Богат, как Спада» И он и молва жили за счет этих пресловутых богатств. Его дворец \nбыл раем для меня. Я учил его племянников, которые потом скончались, и когда он остался один на свете, \nто я отплатил ему беззаветной преданностью за все, что он для меня сделал в продолжение десяти лет.\n\nВ доме кардинала от меня не было тайн; не раз видел я, как он усердно перелистывает старинные книги и \nжадно роется в пыли фамильных рукописей. Когда я как-то упрекнул его за бесполезные бессонные ночи, \nпосле которых он впадал в болезненное уныние, он взглянул на меня с горькой улыбкой и раскрыл передо мною \nисторию города Рима. В этой книге, в двадцатой главе жизнеописания папы 

In [21]:
inputs = gen_model.tokenizer(f'{text} summary:', max_length=512, padding='max_length', truncation=True, return_tensors="pt")
for i in range(100):    
    outputs = gen_model.model.generate(input_ids=inputs["input_ids"].to(device="cuda"), attention_mask=inputs["attention_mask"].to(device="cuda"))
    pred = gen_model.tokenizer.decode(outputs[0], skip_special_tokens=True)
    inputs = gen_model.tokenizer(pred, max_length=512, padding='max_length', truncation=True, return_tensors="pt")

print(pred)

– Вам известно, – сказал аббат, – что я был секретарем, доверенным другом кардинала Спада, 
последнего представителя древнего рода. Этому достойному вельможе я обязан всем счастьем, которое 
я знал в жизни. Он не был богат, хотя богатства его рода стали притчей во языцех, и мне часто приходилось 
слышать выражение: «Богат, как Спада» И он и молва жили за счет этих пресловутых богатств. Его дворец 
был раем для меня. Я учил его племянников, которые потом скончались, и когда он остался один на свете, 
то я отплатил ему беззаветной преданностью за все, что он для меня сделал в продолжение десяти лет.

В доме кардинала от меня не было тайн; не раз видел я, как он усердно перелистывает старинные книги и 
жадно роется в пыли фамильных рукописей. Когда я как-то упрекнул его за бесполезные бессонные ночи, 
после которых он впадал в болезненное уныние, он взглянул на меня с горькой улыбкой и раскрыл передо мною 
историю города Рима. В этой книге, в двадцатой главе жизнеописания папы Александра 